In [54]:
import mediapipe as mp
import pickle as pk
from tqdm import tqdm
import cv2
import os
import pickle
import datetime as dt

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### ISL Dataset

In [55]:
isl_dir = './data/isl_data/'
print(f'Number of signs included: {len(os.listdir(isl_dir))}')
isl_signs = os.listdir(isl_dir)
isl_signs.sort()
for d in isl_signs:
    if d == '.DS_Store':
        continue
    print(f'{d}: {len(os.listdir(os.path.join(isl_dir, d)))}')

Number of signs included: 36
1: 1200
2: 1200
3: 1200
4: 1200
5: 1200
6: 1200
7: 1200
8: 1200
9: 1200
A: 1200
B: 1200
C: 1200
D: 1200
E: 1200
F: 1200
G: 1200
H: 1200
I: 1200
J: 1200
K: 1200
L: 1200
M: 1200
N: 1200
O: 1200
P: 1200
Q: 1200
R: 1200
S: 1200
T: 1200
U: 1200
V: 1200
W: 1200
X: 1200
Y: 1200
Z: 1200


### Creating labelled dataset

In [56]:
# initializing image directory

DATA_DIR = './data/isl_data'
data_prefix = f'prep_data/{DATA_DIR.split("/")[-1]}'

In [57]:
# initiating mediapipe configurations

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# 1-hand vs 2-hands signs
signs_1_hand = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'C', 'I', 'L', 'O', 'U', 'V']
signs_2_hand = ['A', 'B', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'W', 'X', 'Y', 'Z']

In [58]:
data1 = []
labels1 = []
data2 = []
labels2 = []
max_count = 20

for dir_ in tqdm(os.listdir(DATA_DIR)):
    if dir_ == '.DS_Store':
        continue
    count = 0
    for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
        data_aux = []
        img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x)
                    data_aux.append(y)
            if len(data_aux) == 42 and dir_ in signs_1_hand:
                data1.append(data_aux)
                labels1.append(dir_)
                count += 1
            elif len(data_aux) == 84 and dir_ in signs_2_hand:
                data2.append(data_aux)
                labels2.append(dir_)
                count += 1

        if count == max_count:
            break

100%|██████████| 36/36 [01:54<00:00,  3.17s/it]


In [59]:
now = dt.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
f = open(f'{data_prefix}_{now}.pickle', 'wb')
pickle.dump({'data1': data1, 'labels1': labels1, 'data2': data2, 'labels2': labels2}, f)
print(f'Dataset stored at {data_prefix}_{now}.pickle')
f.close()

Dataset stored at prep_data/isl_data_2023_10_21_13_01_38.pickle


### Training a model - Random Forest Classifier (Scikit-learn)

In [61]:
# initializing model directory

DATA_LOC = 'prep_data/isl_data_2023_10_21_13_01_38.pickle'
model_prefix = f'trained_models/{DATA_LOC.split("/")[-1].split("_")[0]}_model'

# loading labelled dataset

data_dict = pickle.load(open(DATA_LOC, 'rb'))

In [62]:
# initializing data & label variables

data1 = np.asarray(data_dict['data1'])
labels1 = np.asarray(data_dict['labels1'])
data2 = np.asarray(data_dict['data2'])
labels2 = np.asarray(data_dict['labels2'])

In [63]:
def train_test_pipeline(data, labels):
    
    # train-test split
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    # training Random Forest Classifier model
    clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    print('Training...')
    clf.fit(X_train, y_train)

    # testing the trained model

    print('Testing...')
    y_pred = clf.predict(X_test)
    print(f'{accuracy_score(y_test, y_pred) * 100}% of samples classified correctly')

    return clf

In [64]:
print('Model for 1-hand signs:')
model1 = train_test_pipeline(data=data1, labels=labels1)
print('\nModel for 2-hands signs:')
model2 = train_test_pipeline(data=data2, labels=labels2)

Model for 1-hand signs:
Training...
Testing...
100.0% of samples classified correctly

Model for 2-hands signs:
Training...
Testing...
96.05263157894737% of samples classified correctly


In [65]:
# storing the model

now = dt.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
f = open(f'{model_prefix}_{now}.pickle', 'wb')
pickle.dump({'model1': model1, 'model2': model2}, f)
f.close()
print(f'Model stored at {model_prefix}_{now}.pickle')

Model stored at trained_models/isl_model_2023_10_21_13_02_04.pickle
